# Portfolio LONG ONLY

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

import random
import ast

import sys

In [3]:
data= pd.read_csv("../data/return.csv")
sp= pd.read_csv("../data/sp500.csv")
portfolio= pd.read_csv("../data/portfolio_constitution.csv")
return_predict_month= pd.read_excel("../data/return_predict_monthly.xlsx")

In [4]:
return_predict_month["Returns_predict"]= return_predict_month["Returns_predict"].apply(ast.literal_eval)
return_predict_month.head()

,Asset,Model,Returns_predict
0,HES US Equity,<catboost.core.CatBoostRegressor object at 0x7...,"[-2.832813166301726, 1.0798846506686033, -3.68..."
1,NEM US Equity,RandomForestRegressor,"[-0.9924100669999997, -0.8786505200000004, -4...."
2,INTC US Equity,RandomForestRegressor,"[-0.2536885459999996, -2.2266463759999984, 0.0..."
3,IFF US Equity,RandomForestRegressor,"[2.5741313260000025, 2.865933291000004, 1.6514..."
4,MOS US Equity,XGBRegressor,"[-0.2642870545387268, -8.006417274475098, 7.39..."


In [5]:
sp["return"]= 100 * (sp["Close"].shift(-1) - sp["Close"])/sp["Close"]
sp["return"]= sp["return"].shift(1)
sp_return= sp[["Date", "return"]].dropna()
sp_return.head()

,Date,return
1,2010-01-01,-3.697425
2,2010-02-01,2.851369
3,2010-03-01,5.879643
4,2010-04-01,1.475923
5,2010-05-01,-8.197584


In [6]:
sp.head()

,Date,Open,High,Low,Close,Adj Close,Volume,return
0,2009-12-01,1098.890015,1130.380005,1085.890015,1115.099976,1115.099976,89515330000,NaN
1,2010-01-01,1116.560059,1150.449951,1071.589966,1073.869995,1073.869995,90947580000,-3.697425
2,2010-02-01,1073.890015,1112.420044,1044.500000,1104.489990,1104.489990,84561340000,2.851369
3,2010-03-01,1105.359985,1180.689941,1105.359985,1169.430054,1169.430054,103683550000,5.879643
4,2010-04-01,1171.229980,1219.800049,1170.689941,1186.689941,1186.689941,116741910000,1.475923


In [9]:
testok= sp["Close"].values
testok[0]/testok[-1] + 1

1.3550165817994166

In [102]:
# transforme le df avec les titres des portefeuilles
portfolio["portfolio"]= portfolio["portfolio"].str.strip('][').str.split(', ')
portfolio["portfolio"]= portfolio["portfolio"].apply(
    lambda row: [x.replace("'", "") for x in row])

portfolio.head()
portfolio["portfolio"][0]

['HES US Equity', 'MO US Equity', 'WELL US Equity']

In [103]:
# calcul cov et var
tickers= [i for i in data.columns if i not in ["Date1", "Date2"]]
data_tickers= data[tickers]

data_corr= data_tickers.corr()
data_cov= data_tickers.cov()


# std= np.std(data_tickers, axis= 0)
# idn= std.index

# std_dict= {idn[i]:std[i] for i in range(len(std))}
# std_dict

In [104]:
test= np.array([[23, 21, 17, 10, 9], [6, 5, 9, 7, 3], [0, -2, -6, 2, 0]])
test
w_test= np.array([[random.random() for _ in range(test.shape[1])] for _ in range(test.shape[0])])
w_test2= w_test/np.sum(w_test, axis= 0)
w_test2

array([[0.02560072, 0.50609691, 0.6028089 , 0.33309068, 0.20500217],
       [0.23339058, 0.00907911, 0.07668915, 0.4438735 , 0.6524501 ],
       [0.74100869, 0.48482399, 0.32050194, 0.22303582, 0.14254773]])

In [105]:
# w_test2.T * le return sur toutes les périodes
w_test2.T[0]

array([0.02560072, 0.23339058, 0.74100869])

In [106]:
# ajout matrice var cov dans le dataset par portefeuille

portfolio["cov"]= portfolio["portfolio"].apply(lambda row: np.matrix(data_cov.loc[row, row]))
# save
portfolio.to_csv("../data/result_portfolio.csv", index= False, header= True)

In [107]:
data_cov.loc[portfolio["portfolio"][1], portfolio["portfolio"][1]]

,TGT US Equity,DUK US Equity,DTE US Equity,HSY US Equity,WEC US Equity
TGT US Equity,43.578263,-2.287598,-0.485471,1.547108,1.067696
DUK US Equity,-2.287598,15.092307,12.000453,6.921832,13.998862
DTE US Equity,-0.485471,12.000453,14.839588,8.117057,13.855017
HSY US Equity,1.547108,6.921832,8.117057,24.238535,8.163991
WEC US Equity,1.067696,13.998862,13.855017,8.163991,18.955652


In [108]:
w_test= np.array([[random.random() for _ in range(test.shape[1])] for _ in range(test.shape[0])])
w_test2= w_test/np.sum(w_test, axis= 0)
w_test2

array([[0.39715045, 0.1808522 , 0.11928259, 0.29924551, 0.0176572 ],
       [0.17312068, 0.11217693, 0.30357425, 0.34203537, 0.32723007],
       [0.42972887, 0.70697087, 0.57714316, 0.35871912, 0.65511273]])

In [109]:
def poids_opti(row0, row1):
    stock_var_portfolio= {}
    for i in range(1000):
    #     random.seed(55)
#         cov_test= portfolio["cov"][1]
        cov_test= row1
        test= np.array([random.random() for _ in range(len(row0))])
        test= test/np.sum(test)
        res_var= np.asarray(np.dot(np.dot(test.T, cov_test), test))[0][0]
        stock_var_portfolio[res_var]= test
    return min(stock_var_portfolio.items(), key=lambda x: x[0])

In [110]:
## stock_var_portfolio
ok= poids_opti(portfolio["portfolio"][1], portfolio["cov"][1])
ok
# test= np.array([random.random() for _ in range(len(row0))])
# test= test/np.sum(test)
# res_var= np.asarray(np.dot(np.dot(test.T, cov_test), test))[0][0]

(9.189516852268875,
 array([0.20272742, 0.35041467, 0.28551667, 0.14525224, 0.01608899]))

In [111]:
portfolio["poids_var"]= portfolio.apply(lambda row: poids_opti(row[0], row[1])
                                       , axis= 1)

In [112]:
# # save
# portfolio.to_csv("../data/result_portfolio_weights.csv", index= False, header= True)

# portfolio.head()

,portfolio,cov,poids_var
0,"[HES US Equity, MO US Equity, WELL US Equity]",[[[[[100.41903311 6.72056771 4.09955939]]]...,"(15.969328981372433, [0.11594522554395133, 0.4..."
1,"[TGT US Equity, DUK US Equity, DTE US Equity, ...",[[[[[43.5782631 -2.28759833 -0.48547121 1.54...,"(9.20021268755812, [0.220932843624211, 0.40894..."
2,"[PPL US Equity, MO US Equity, DTE US Equity]","[[[[[19.51266423 6.70958138 9.18255513]]]], ...","(11.444908619953848, [0.2864190230561245, 0.21..."
3,"[KMB US Equity, PPL US Equity, WEC US Equity]","[[[[[18.98891434 6.79175117 8.30634734]]]], ...","(11.77000920810331, [0.36815131523404365, 0.34..."
4,"[HES US Equity, MO US Equity, DTE US Equity, W...",[[[[[100.41903311 6.72056771 4.50376371 ...,"(11.678257299587885, [0.07638415255470991, 0.2..."


In [1]:
# predicted_returns= return_predict_month["Returns_predict"].values

size= len(return_predict_month["Returns_predict"][0])

predicted_returns= {}
for i in range(return_predict_month.shape[0]):
    get_data= return_predict_month[["Asset", "Returns_predict"]]
    predicted_returns[get_data["Asset"][i]]= get_data["Returns_predict"][i]

NameError: name 'return_predict_month' is not defined

In [114]:
def matrix_return_(port, returns_= predicted_returns):
    test= port
    ass_return= []
    for ass in test:
        ass_return.append(returns_[ass])

    return np.array(ass_return)

In [115]:
portfolio["return_predicted"]= portfolio["portfolio"].apply(lambda row:
                                                            matrix_return_(row))

In [116]:
# # save
# portfolio.to_csv("../data/result_portfolio_weights_returnp.csv", index= False, header= True)

In [117]:
data_effectif= data.iloc[data.shape[0]-size:, :48]

ass_return= {}
for ass in data_effectif.columns:
    ass_return[ass]= [val for val in data_effectif[ass].values]
ass_return

def matrix_return_(port, returns_= ass_return):
    test= port
    ass_return= []
    for ass in test:
        ass_return.append(returns_[ass])

    return np.array(ass_return)

portfolio["return_effectif"]= portfolio["portfolio"].apply(lambda row:
                                                            matrix_return_(row))

In [118]:
# # save
# portfolio.to_csv("../data/result_portfolio_weights_returnp_e.csv", index= False, header= True)

# portfolio.head()

,portfolio,cov,poids_var,return_predicted,return_effectif
0,"[HES US Equity, MO US Equity, WELL US Equity]",[[[[[100.41903311 6.72056771 4.09955939]]]...,"(15.969328981372433, [0.11594522554395133, 0.4...","[[-2.832813166301726, 1.0798846506686033, -3.6...","[[-1.9668889999999999, 0.8801783999999999, -8...."
1,"[TGT US Equity, DUK US Equity, DTE US Equity, ...",[[[[[43.5782631 -2.28759833 -0.48547121 1.54...,"(9.20021268755812, [0.220932843624211, 0.40894...","[[0.3509575223380869, -0.08107992137471835, -0...","[[-4.833492, -1.819636, 1.376736, -5.289723, 9..."
2,"[PPL US Equity, MO US Equity, DTE US Equity]","[[[[[19.51266423 6.70958138 9.18255513]]]], ...","(11.444908619953848, [0.2864190230561245, 0.21...","[[-0.2815525667233901, -0.1870743011434873, 0....","[[0.8521866, 8.418972, 2.770689, -0.002969211,..."
3,"[KMB US Equity, PPL US Equity, WEC US Equity]","[[[[[18.98891434 6.79175117 8.30634734]]]], ...","(11.77000920810331, [0.36815131523404365, 0.34...","[[0.45476294521461735, 0.7550758705277701, 1.9...","[[0.11973620000000001, 1.210357, 3.390857, -1...."
4,"[HES US Equity, MO US Equity, DTE US Equity, W...",[[[[[100.41903311 6.72056771 4.50376371 ...,"(11.678257299587885, [0.07638415255470991, 0.2...","[[-2.832813166301726, 1.0798846506686033, -3.6...","[[-1.9668889999999999, 0.8801783999999999, -8...."


In [119]:
portfolio["var"]= portfolio["poids_var"].apply(lambda row: row[0])
portfolio["poids_var"]= portfolio["poids_var"].apply(lambda row: row[1]).apply(lambda row: np.array(row))

In [120]:
portfolio.head()

,portfolio,cov,poids_var,return_predicted,return_effectif,var
0,"[HES US Equity, MO US Equity, WELL US Equity]",[[[[[100.41903311 6.72056771 4.09955939]]]...,"[0.11594522554395133, 0.44304832744179684, 0.4...","[[-2.832813166301726, 1.0798846506686033, -3.6...","[[-1.9668889999999999, 0.8801783999999999, -8....",15.969329
1,"[TGT US Equity, DUK US Equity, DTE US Equity, ...",[[[[[43.5782631 -2.28759833 -0.48547121 1.54...,"[0.220932843624211, 0.4089450400752729, 0.1127...","[[0.3509575223380869, -0.08107992137471835, -0...","[[-4.833492, -1.819636, 1.376736, -5.289723, 9...",9.200213
2,"[PPL US Equity, MO US Equity, DTE US Equity]","[[[[[19.51266423 6.70958138 9.18255513]]]], ...","[0.2864190230561245, 0.21257649826609343, 0.50...","[[-0.2815525667233901, -0.1870743011434873, 0....","[[0.8521866, 8.418972, 2.770689, -0.002969211,...",11.444909
3,"[KMB US Equity, PPL US Equity, WEC US Equity]","[[[[[18.98891434 6.79175117 8.30634734]]]], ...","[0.36815131523404365, 0.34206628075784323, 0.2...","[[0.45476294521461735, 0.7550758705277701, 1.9...","[[0.11973620000000001, 1.210357, 3.390857, -1....",11.770009
4,"[HES US Equity, MO US Equity, DTE US Equity, W...",[[[[[100.41903311 6.72056771 4.50376371 ...,"[0.07638415255470991, 0.2792108681164098, 0.39...","[[-2.832813166301726, 1.0798846506686033, -3.6...","[[-1.9668889999999999, 0.8801783999999999, -8....",11.678257


In [156]:
portfolio["portfolio_return"]= portfolio[["poids_var", "return_predicted"]].apply(lambda row:
                                                                                  np.dot(row[0], row[1])
                                                                                 , axis= 1)
portfolio["portfolio_return_effectif"]= portfolio[["poids_var", "return_effectif"]].apply(lambda row:
                                                                                  np.dot(row[0], row[1])
                                                                                 , axis= 1)

# compute sharpe ratio (c'est pas vraiment un sharpe ratio)
portfolio["relatif_return"]= portfolio[["portfolio_return", "var"]].apply(lambda row:
                                                                         row[0]/row[1]
                                                                         , axis= 1)

In [157]:
portfolio.head()

,portfolio,cov,poids_var,return_predicted,return_effectif,var,portfolio_return,portfolio_return_effectif,relatif_return
0,"[HES US Equity, MO US Equity, WELL US Equity]",[[[[[100.41903311 6.72056771 4.09955939]]]...,"[0.11594522554395133, 0.44304832744179684, 0.4...","[[-2.832813166301726, 1.0798846506686033, -3.6...","[[-1.9668889999999999, 0.8801783999999999, -8....",15.969329,"[1.8778578802802723, 2.5288091337603777, 0.787...","[1.7999052478901443, 2.599222273274183, 1.2120...","[0.11759153327423566, 0.1583541260068053, 0.04..."
1,"[TGT US Equity, DUK US Equity, DTE US Equity, ...",[[[[[43.5782631 -2.28759833 -0.48547121 1.54...,"[0.220932843624211, 0.4089450400752729, 0.1127...","[[0.3509575223380869, -0.08107992137471835, -0...","[[-4.833492, -1.819636, 1.376736, -5.289723, 9...",9.200213,"[1.2435394621600115, 1.059170592822048, 0.7516...","[0.8615075562136874, 2.5758650440608157, 0.666...","[0.1351642080885487, 0.1151245768757514, 0.081..."
2,"[PPL US Equity, MO US Equity, DTE US Equity]","[[[[[19.51266423 6.70958138 9.18255513]]]], ...","[0.2864190230561245, 0.21257649826609343, 0.50...","[[-0.2815525667233901, -0.1870743011434873, 0....","[[0.8521866, 8.418972, 2.770689, -0.002969211,...",11.444909,"[1.7127189253155728, 1.5640557688693852, 0.462...","[3.7460668470097276, 4.679418567993872, 2.8405...","[0.14964898210978284, 0.1366595244056822, 0.04..."
3,"[KMB US Equity, PPL US Equity, WEC US Equity]","[[[[[18.98891434 6.79175117 8.30634734]]]], ...","[0.36815131523404365, 0.34206628075784323, 0.2...","[[0.45476294521461735, 0.7550758705277701, 1.9...","[[0.11973620000000001, 1.210357, 3.390857, -1....",11.770009,"[0.6517514637968083, 0.8196678459965827, 1.483...","[1.2166803309676266, 4.000016436333115, 2.4923...","[0.055373912821418715, 0.06964037423456433, 0...."
4,"[HES US Equity, MO US Equity, DTE US Equity, W...",[[[[[100.41903311 6.72056771 4.50376371 ...,"[0.07638415255470991, 0.2792108681164098, 0.39...","[[-2.832813166301726, 1.0798846506686033, -3.6...","[[-1.9668889999999999, 0.8801783999999999, -8....",11.678257,"[2.0305311314458296, 2.2498096991700693, 0.570...","[3.5201052487549447, 2.809282986360321, 1.6747...","[0.17387278592649993, 0.1926494374506942, 0.04..."


In [200]:
print(np.stack(np.array(portfolio["relatif_return"])).shape)
relatif_return_matrix= np.stack(np.array(portfolio["relatif_return"]))
predicted_return_matrix= np.stack(np.array(portfolio["portfolio_return"]))
effectif_return_matrix= np.stack(np.array(portfolio["portfolio_return_effectif"]))



np.argmax(relatif_return_matrix, axis= 0)

# get la position du portefeuille avec le return relatif le plus grand
which_portfolio= relatif_return_matrix.max(axis=0,keepdims=1) == relatif_return_matrix

(9600, 105)


In [208]:
np.set_printoptions(threshold=sys.maxsize)
which_return_pred= which_portfolio * predicted_return_matrix
which_return_eff= which_portfolio * effectif_return_matrix

which_return_pred

3.991765956270785

In [219]:
position= np.argwhere(which_return_pred != 0)
position= position[np.argsort(position[:, 1])]
np.save("../data/position_portfolio.npy", position)
position_saved= np.load("../data/position_portfolio.npy")
position_saved

array([[ 463,    0],
       [2167,    1],
       [8129,    2],
       [6064,    3],
       [3191,    4],
       [ 811,    5],
       [3296,    6],
       [7719,    7],
       [9415,    8],
       [ 571,    9],
       [9434,   10],
       [4328,   11],
       [ 500,   12],
       [3461,   13],
       [4260,   14],
       [3148,   15],
       [ 552,   16],
       [1771,   17],
       [2794,   18],
       [8120,   19],
       [5039,   20],
       [2306,   21],
       [4214,   22],
       [4283,   23],
       [2794,   24],
       [3270,   25],
       [2794,   26],
       [7644,   27],
       [4232,   28],
       [2391,   29],
       [3663,   30],
       [6863,   31],
       [7629,   32],
       [7324,   33],
       [ 479,   34],
       [4755,   35],
       [4253,   36],
       [8380,   37],
       [4336,   38],
       [7108,   39],
       [3259,   40],
       [2606,   41],
       [8286,   42],
       [6011,   43],
       [4336,   44],
       [8255,   45],
       [7431,   46],
       [1808,

In [228]:
eff_return= which_return_eff[position_saved[:,0], position_saved[:,1]]
pred_return= which_return_pred[position_saved[:,0], position_saved[:,1]]

In [233]:
sp_return_only= sp_return.iloc[data.shape[0]-size:, 1].values
date= sp_return.iloc[data.shape[0]-size:, 0].values

In [234]:
all_data_long_only= pd.DataFrame({
    "sp500":sp_return_only
    , "effectif":eff_return
    , "prediction":pred_return
    , "date": date
})
all_data_long_only.head()

,sp500,effectif,prediction,date
0,-0.104731,5.393158,3.697888,2011-03-01
1,2.849538,3.593613,3.375629,2011-04-01
2,-1.350095,0.496853,2.915016,2011-05-01
3,-1.825746,0.721248,3.057459,2011-06-01
4,-2.147443,-1.418650,3.848040,2011-07-01


In [235]:
# save
all_data_long_only.to_csv("../data/long_only_return_.csv", index= False, header= True)